### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.sales-salesorderheader"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
sales_salesorderheader = df_processados['sales_salesorderheader']

print(f"Colunas: {sales_salesorderheader.shape[1]}\nLinhas: {sales_salesorderheader.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'salesorderid'
duplicadas = sales_salesorderheader[
    sales_salesorderheader.duplicated(subset=['salesorderid'], keep=False)
]

# Verificar se existem duplicadas
if not duplicadas.empty:
    # Ordenar duplicadas por 'salesorderid' e 'modifieddate'
    duplicadas_ordenadas = duplicadas.sort_values(by=['salesorderid', 'modifieddate'])

    # Exibir duplicadas ordenadas
    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

sales_salesorderheader = sales_salesorderheader.drop_duplicates(subset=['salesorderid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(sales_salesorderheader)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = sales_salesorderheader.copy()


#doc: bkp = backup (cópia)

In [ ]:
# Ordenar e exibir o df por 'salesorderid'
sales_salesorderheader = sales_salesorderheader.sort_values(by=['salesorderid'])

print(sales_salesorderheader)

In [ ]:
sales_salesorderheader.info()

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in sales_salesorderheader.columns:   
    missing_rows = sales_salesorderheader[sales_salesorderheader[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")



#doc: algumas colunas apresentam valores ausentes:      

# purchaseordernumber: 27659 linhas ausentes, aprox. 88% de dados ausentes - coluna deletada
# salespersonid: 27659 linhas ausentes, aprox. 88% de dados ausentes - coluna deletada
# creditcardid: 1131 linhas ausentes, mantive a coluna por ser ID Coluna 
# creditcardapprovalcode: 1131 linhas ausentes, mantive a coluna
# currencyrateid: 17489 linhas ausentes, aprox. 56% de dados ausentes - coluna deletada
# comment: 31465 linhas ausentes, 100% de dados ausentes - coluna deletada         
 

In [ ]:
#deletar colunas com muitos valores ausentes
sales_salesorderheader = sales_salesorderheader.drop(columns=['purchaseordernumber', 'salespersonid', 'currencyrateid', 'comment'])

print("Colunas restantes após remoção:")
print(sales_salesorderheader.columns)

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = sales_salesorderheader.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)



In [ ]:
#verificar informações do df
sales_salesorderheader.info()

In [ ]:
#avaliando as variáveis qualitativas*

# Lista de colunas qualitativas para análise
coluna_qualitativa = ["status", "onlineorderflag", "territoryid", "shipmethodid"]

# Análise de valores para cada coluna qualitativa
for col in coluna_qualitativa:
    counts = sales_salesorderheader[col].value_counts().nlargest(10) 
    percentages = (counts / sales_salesorderheader.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    print(f"Resumo para a coluna '{col}':")
    print(summary)
    print()



#doc*: variáveis qualitativas são um tipo de variável estatística que representam características ou atributos dos dados, sem serem medidas numericamente
#      no nosso caso, jobtitle, maritalstatus, gender, por exemplo

In [ ]:
# Agrupar por 'territoryid' para calcular quantidade e proporção de pedidos online
territory_summary = sales_salesorderheader.groupby('territoryid')['onlineorderflag'].agg(
    qtde='count',
    online_percent=lambda x: x.mean() * 100
)

# Formatando a coluna de porcentagem
territory_summary['%'] = territory_summary['online_percent'].map("{:.2f}%".format)

# Selecionando as colunas desejadas e ordenando pela maior quantidade
territory_summary = territory_summary[['qtde', '%']].sort_values(by='qtde', ascending=False)

# Exibindo o resumo
print("Resumo por território (quantidade e % de pedidos online):")
print(territory_summary)


In [ ]:
# Análise cruzada entre 'territoryid' e 'shipmethodid'
territory_ship = sales_salesorderheader.groupby(['territoryid', 'shipmethodid']).size().unstack(fill_value=0)
print("Distribuição de métodos de envio por território:")
print(territory_ship)

# Visualização
territory_ship.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='viridis')
plt.title("Métodos de Envio por Território")
plt.xlabel("Território")
plt.ylabel("Quantidade de Pedidos")
plt.show()


In [ ]:
# Agrupar por 'onlineorderflag' e 'shipmethodid'
online_ship = sales_salesorderheader.groupby(['onlineorderflag', 'shipmethodid']).size().unstack(fill_value=0)
print("Métodos de envio por tipo de pedido (online/offline):")
print(online_ship)

# Visualização
online_ship.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='coolwarm')
plt.title("Métodos de Envio por Tipo de Pedido")
plt.xlabel("Pedido Online (True/False)")
plt.ylabel("Quantidade de Pedidos")
plt.show()


In [ ]:
# Receita total por território
territory_revenue = sales_salesorderheader.groupby('territoryid')['subtotal'].sum().sort_values(ascending=False)
print("Receita total por território:")
print(territory_revenue)

# Visualização
territory_revenue.plot(kind='bar', figsize=(12, 6), color='purple', edgecolor='black')
plt.title("Receita Total por Território")
plt.xlabel("Território")
plt.ylabel("Subtotal (Receita Total)")
plt.show()


In [ ]:
# Variáveis quantitativas*: estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['subtotal', 'taxamt','freight', 'totaldue']

# Estatísticas Descritivas das colunas numéricas*
print(sales_salesorderheader[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = sales_salesorderheader[col].quantile(0.25)
    q3 = sales_salesorderheader[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = sales_salesorderheader[(sales_salesorderheader[col] < lower_bound) | (sales_salesorderheader[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas, no nosso caso, vacationhours, sickleavehours, por exemplo
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados

#doc****: as colunas analisadas apresentam outliers, os outliers identificados parecem legítimos no contexto do AdventureWorks, refletindo compras em grande volume, 
#         produtos premium ou custos logísticos elevados. Resultando em não manipular tais colunas

In [ ]:
# Configurar o estilo do gráfico
sns.set(style='dark', rc={"axes.facecolor": "black", "figure.facecolor": "black"})

# Lista das colunas para análise
columns_to_plot = ['subtotal', 'taxamt', 'freight', 'totaldue']

# Criar histogramas (1 por linha)
for col in columns_to_plot:
    plt.figure(figsize=(12, 6))
    
    # Histograma
    ax = sns.histplot(
        sales_salesorderheader[col], bins=30, kde=False, color="blue", edgecolor="white"
    )
    
    # Título e rótulos
    plt.title(f'Distribuição de {col}', color='white', fontsize=16)
    plt.xlabel(col, color='white', fontsize=14)
    plt.ylabel("Frequência", color='white', fontsize=14)
    
    # Cor dos rótulos
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    
    # Remover grade
    ax.grid(False)
    
    # Exibir o gráfico
    plt.show()


In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['sales_salesorderheader'] = sales_salesorderheader

In [ ]:
sales_salesorderheader.info()

In [ ]:
# Padronizar colunas com valores textuais
sales_salesorderheader['rowguid'] = sales_salesorderheader['rowguid'].str.strip().str.upper()

print(sales_salesorderheader.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")